In [ ]:
# module and library required to build a Face Recognition System
import face_recognition
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import os
import pickle

from PIL import Image

In [ ]:
known_images = []
imgs = []
known_encodings = []
known_names = []

In [ ]:
try:
    data = pickle.loads(open("encodings.pickle", "rb").read())
    known_encodings = data["encodings"]
    known_names = data["names"]
except:
    print("No encodings to load")

In [ ]:
PATH_KNOWN = "known_preview/"
ind_real = 0
for ind, p_known_image in enumerate(os.listdir(PATH_KNOWN)):
    potential_name = p_known_image[:-6]
    if potential_name not in known_names:
        p_known_image = PATH_KNOWN + p_known_image

        # if want to use known_images then need to pop last if face encoding isnt added
        known_images.append(face_recognition.load_image_file(p_known_image))
        imgs.append(cv2.imread(p_known_image))
        plt.figure(figsize=(15,15))
        plt.imshow(imgs[ind_real])
        plt.show()
        try:
            known_encodings.append(face_recognition.face_encodings(known_images[ind_real])[0])
            known_names.append(potential_name)
        except:
            print("Didn't add face from {}".format(p_known_image))
        finally:
            ind_real += 1
    else:
        # if wanted to add new encodings for same person then what?
        pass

In [ ]:
data = {"encodings": known_encodings, "names": known_names}
f = open("encodings.pickle", "wb")
f.write(pickle.dumps(data))
f.close()

In [ ]:
def pickle_unknowns(unk_enc, unk_nam, unk_img, unk_hits):
    unk_data = {'encodings': unk_enc,
               'names': unk_nam,
               'images': unk_img,
               'hits': unk_hits}
    f = open('unknowns.pickle', 'wb')
    f.write(pickle.dumps(unk_data))
    f.close()

In [ ]:
import random;

In [ ]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []

unknown_encodings = []
unknown_names = []
unknown_images = []
unknown_hits = []

unk_dump_counter = 0

PATH_TEST = "../test_set/LETA_train_full_2019-06-01_2019-08-31/LETA_pics_full/"

# for p_test_image in random.shuffle(os.listdir(PATH_TEST)):
test_paths = os.listdir(PATH_TEST)
random.shuffle(test_paths)
for p_test_image in test_paths:
    show_pic = False
    test_show = False
    
    p_test_image = PATH_TEST + p_test_image
    
#     rgb_frame = Image.open(p_test_image)
#     rgb_frame.thumbnail((2000, 2000))
    
    # image = face_recognition.load_image_file(p_test_image)
    
    # can i replace following with the face_rec method above?
    frame = cv2.imread(p_test_image)
    frame = cv2.resize(frame, None, fx=0.4, fy=0.4, interpolation=cv2.INTER_AREA)
    rgb_frame = frame[:, :, ::-1]
    
    face_locations = face_recognition.face_locations(rgb_frame)
    # face_locations = face_recognition.face_locations(rgb_frame, model = "cnn")
    # face_locations_unk = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    face_names = []
    for idx, face_encoding in enumerate(face_encodings):
        match = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=0.50)
        match_unk = face_recognition.compare_faces(unknown_encodings, face_encoding, tolerance=0.50)
        name = None
        for ind, m in enumerate(match):
            if m:
                name = known_names[ind]
                show_pic = True
        for ind, m in enumerate(match_unk):
            if m:
                name = unknown_names[ind]
                unknown_hits[ind] += 1
                # only show pic if we "know" the person
                if unknown_hits[ind] > 4:
                    show_pic = True
                # save unknown faces
                top, right, bottom, left = face_locations[idx]
                face_array = rgb_frame[top:bottom, left:right]
                face_image = Image.fromarray(face_array)
                unknown_images[ind] += [face_image]
                # pickle the unknowns, to use if crashes or sht
                unk_dump_counter += 1
                if unk_dump_counter%10 == 0:
                    pickle_unknowns(unknown_encodings, unknown_names, unknown_images, unknown_hits)
                
        face_names.append(name)
        if not show_pic:
            # save unknown faces
            top, right, bottom, left = face_locations[idx]
            face_array = rgb_frame[top:bottom, left:right]
            face_image = Image.fromarray(face_array)
            
            unknown_images.append([face_image])
            # 2do: same list thing for encodings, for when first pic poor (altho mby explodes)
            unknown_encodings.append(face_encoding)
            unknown_names.append("unk{}".format(len(unknown_names)))
            unknown_hits.append(1)
            # can add something that saves all pics of same unk face
    
    for (top, right, bottom, left), name in zip(face_locations, face_names):
#         if not name:
#             continue
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom + 25), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom + 19), font, 2.0, (255, 255, 255), 3)
    if show_pic:
        plt.figure(figsize = (15, 15))
        plt.imshow(frame)
        plt.show()
        cv2.destroyAllWindows()
        

In [ ]:
# add code that outputs unknown faces in order of hits
# zip hits and face img, output in order?
# good enough if we want human to recog and just add that person

# OR filter out just those that have more than x hits

print(unknown_hits)

for ind, hits in enumerate(unknown_hits):
    if hits > 5:
        print(hits)
        plt.figure(figsize = (15, 15))
        for unk_img in unknown_images[ind]:
            plt.imshow(unk_img)
            plt.show()
            cv2.destroyAllWindows()
        # unknown_images[ind].show()

In [ ]:
# LOAD UNKNOWNS FROM FILE
if True:
    try:
        unk_data = pickle.loads(open("unknowns.pickle", "rb").read())
        unknown_encodings = unk_data["encodings"]
        unknown_names = unk_data["names"]
        unknown_images = unk_data['images']
        unknown_hits = unk_data['hits']
    except:
        print("No encodings to load")

In [ ]:
%notebook -e emerg_save.ipynb